In [1]:
!pip install AugmentedSocialScientist, pandas

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 25.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 43.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 46.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.1/785.1 kB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6

In [1]:
from torch import cuda

cuda.get_device_name(0)

'Tesla T4'

In [2]:
import pandas as pd

annotations_file = pd.read_csv("../data/intermediate/current_annotation_file.csv")
annotations_file.sample(5)

,sentence_id,Personne,sentence_text
983,1300557,NaN,Cellesci se définissent classiquement par le s...
336,140400,Agent responsable,Le droit de ne pas savoir\nIl n'est donc pas é...
415,690037,Personne relationnelle,d) La sensibilisation du corps médical au devo...
1029,80390,NaN,Cette approche commence à être utilisée en agr...
1035,1210153,NaN,Ainsi toutes les \ndemandes de suicide assisté...


In [3]:
annotations = annotations_file.dropna()
annotations.sample(5)

,sentence_id,Personne,sentence_text
598,1060183,Individu,"Par ce type de \nconfiscation, des valeurs éth..."
287,540679,Genre/espèce humaine,"Or, si la reproduction des êtres humains par c..."
28,430507,Genre/espèce humaine,Mais on peut certainement\nprogresser vers une...
85,1370303,Genre/espèce humaine,Intégrer la santé publique aux biens communs (...
500,1110187,Agent responsable,"C’est, dans l’histoire de l’humanité, \nune pr..."


In [4]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.67)

In [5]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: Tesla T4


In [6]:
train_loader = bert.encode(train_data.sentence_text.values,
train_data.Personne.values)

  0%|          | 0/466 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (620 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/466 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [7]:
test_loader = bert.encode(test_data.sentence_text.values,
test_data.Personne.values)

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6}


In [8]:
scores = bert.run_training(
    train_loader,
    test_loader,
    n_epochs=2,
    lr=5e-5,
    random_state=1968,
    save_model_as='clickbait'
    )

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/mamba/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 2 ========
Training...


OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 14.58 GiB of which 305.56 MiB is free. Process 1229085 has 14.28 GiB memory in use. Of the allocated memory 14.08 GiB is allocated by PyTorch, and 80.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
pred_data = annotations_file[annotations_file.isna().any(axis=1)].drop('Personne', axis=1)
pred_loader = bert.encode(pred_data.sentence_text.values)
pred_proba = bert.predict_with_model(pred_loader, model_path='./models/clickbait')
pred_data['pred_label'] = np.argmax(pred_proba, axis=1)
pred_data['pred_proba'] = np.max(pred_proba, axis=1)